In [14]:
import pandas as pd
import sys
import os
import seaborn as sns
import holoviews as hv
from IPython.core.display import display, HTML
from holoviews import opts, dim, Palette

################################ CONFIGURATION FILE HANDLING #################################

import configparser
config = configparser.ConfigParser()

try:
    config.read('config_a.ini')
    # Get values from configuration file
    upper_acceptable_ping_bound = float(config['DEFAULT']['upper_acceptable_ping_bound'])
    upper_ping_issue_bound = float(config['DEFAULT']['upper_ping_issue_bound'])
    acceptable_network_speed = float(config['DEFAULT']['acceptable_network_speed'])
except:
    # In case no config-file is found or another reading error occured
    upper_acceptable_ping_bound = float(10)
    upper_ping_issue_bound = float(99999)
    acceptable_network_speed = float(16)

##############################################################################################

hv.extension('bokeh')

opts.defaults(
    opts.Bars(xrotation=45, tools=['hover']),
    opts.BoxWhisker(width=700, xrotation=30, box_fill_color=Palette('Category20')),
    opts.Curve(width=700, tools=['hover']),
    opts.GridSpace(shared_yaxis=True),
    opts.Scatter(width=700, height=500, color=Palette('Category20'), size=dim('growth')+5, tools=['hover'],alpha=0.5, cmap='Set1'),
    opts.NdOverlay(legend_position='left'))


# Initializes the figures path in webpage for the diagram output
if os.path.isdir("./webpage/figures") == False:
    os.mkdir("./webpage/figures")
    print("Path 'figures' created successfully")
else:
    print("Path 'figures' initialized")

def numOutlierCount(attribute):
    q25=attribute.quantile(0.25)
    q75=attribute.quantile(0.75)
    iqr=q75-q25
    AnzahlMax=attribute[attribute>q75+1.5*iqr].count()
    AnzahlMin=attribute[attribute<q25-1.5*iqr].count()
    AnzahlAusreißerAbsolut=AnzahlMax+AnzahlMin
    AnzahlAusreißerProzentual=AnzahlAusreißerAbsolut/attribute.count()
    print("Ausreißer Absolut: "+str(AnzahlAusreißerAbsolut))
    print("Anzahl Ausreißer prozentual: "+str(AnzahlAusreißerProzentual))
    return(AnzahlAusreißerAbsolut)
def inspect_outliers(df,attribute):
    q25=attribute.quantile(0.25)
    q75=attribute.quantile(0.75)
    iqr=q75-q25
    df_outliers_min=df[attribute<(q25-iqr*1.5)]
    df_outliers_max = df[attribute>(q75+iqr*1.5)]
    display(HTML("</br><h2>Min-Outliers</h2>"))
    display(HTML(df_outliers_min.to_html()))
    display(HTML("<hr></br><h2>Max-Outliers</h2>"))
    display(HTML(df_outliers_max.to_html()))
    
# Data Import
try:
    df_ping = pd.read_csv("Data/ping_test.csv",index_col=0)
    df_speed_test = pd.read_csv("Data/speed_test.csv", index_col=0)

    df_ping["date"] = pd.to_datetime(df_ping["date"],format="%d.%m.%Y %H:%M:%S")
    df_speed_test["date"] = pd.to_datetime(df_speed_test["date"], format="%d.%m.%Y %H:%M:%S")
except:
    print("Error while searching for files. Please perform network-test first.")
    sys.exit(0)

# Basic Data Wrangling

In [15]:
print(df_ping.shape)
df_ping.head()


In [16]:
print(df_speed_test.shape)
df_speed_test.head()

In [17]:
df_ping_issues = df_ping[df_ping["max"]==upper_ping_issue_bound]
print("There are {} issues in the analysis of the ping.".format(df_ping_issues.shape[0]))

df_speed_test_issues = df_speed_test[df_speed_test["ping"]==upper_ping_issue_bound]
print("There are {} issues in the analysis of the network speed.".format(df_speed_test_issues.shape[0]))
# Filter issues from eg. sockets
df_ping = df_ping[df_ping["max"]!=upper_ping_issue_bound]
df_speed_test = df_speed_test[df_speed_test["ping"]!=upper_ping_issue_bound]

In [18]:
df_speed_test["ping"].max()

In [19]:
print(
    "The maximal Ping time has been {} ms.\nThe minimal ping time has been {} ms. \nThe mean ping time has been {} ms. "
    .format(df_ping["max"].max(), df_ping["min"].min(), df_ping["avg"].mean()))

 # Ping Times in ms with extreme outlieres

In [20]:
fig_all_max_ping = hv.Curve((df_ping["date"], df_ping["max"]),
                            "Date",
                            "Ping in ms",
                            label="All messured pings")
fig_dot_over_upper_bound = hv.Scatter(
    (df_ping["date"][df_ping["max"] > upper_acceptable_ping_bound],
     df_ping["max"][df_ping["max"] > upper_acceptable_ping_bound]),
    "Date",
    "Max_Ping_Time",
    label="Highlight pings over {} ms".format(
        str(upper_acceptable_ping_bound))).opts(opts.Scatter(color="red", size=10))

fig_ping_times_with_extreme_outliers = (fig_all_max_ping *
                                        fig_dot_over_upper_bound).opts(
                                            legend_position="top_left",
                                            title="All Max. Ping Times in ms",padding=0.05)
#Safe newly generated plot
hv.save(fig_ping_times_with_extreme_outliers,
        os.path.join("webpage", "figures",
                     "fig_ping_times_with_extreme_outliers.html"),
        backend='bokeh')
fig_ping_times_with_extreme_outliers

In [21]:
inspect_outliers(df_ping,df_ping["max"])

# Ping Times in ms without extreme outlieres

In [22]:
fig_ping_without_extreme_outliers = hv.Curve(
    (df_ping["date"][df_ping["max"]<1000],
     df_ping["max"][df_ping["max"]<1000]),"Date","Ping in ms",label="All ping times less then 1000 ms")

fig_ping_highlight_max = hv.Scatter(
         (df_ping["date"][df_ping["max"] > upper_acceptable_ping_bound][df_ping["max"]<1000],
          df_ping["max"][df_ping["max"] > upper_acceptable_ping_bound][df_ping["max"]<1000]),
         "Date",
         "Max_Ping_Time",
    label = "Highlight pings over {} ms".format(str(upper_acceptable_ping_bound))
     ).opts(color="red", size=10)

fig_ping_times_without_extreme_outliers = (fig_ping_without_extreme_outliers*fig_ping_highlight_max).opts(title="All Max. Ping Times in ms without extreme outlieres",
                                                                                                         legend_position="top_left",
                                                                                                         padding = 0.05)

#Safe newly generated plot
hv.save(fig_ping_times_without_extreme_outliers, os.path.join("webpage","figures","fig_ping_times_without_extreme_outliers.html") , backend='bokeh')
fig_ping_times_without_extreme_outliers

In [23]:
# The latency bound under which network speedtest is performing is defined in the network_test.py

pingbound_network_test = df_speed_test["ping"].min()

fig_network_speed_below_pingbound = hv.Curve(
    (df_speed_test["date"], df_speed_test["downstream"]/1000),
    "Date",
    "Network Speed",
    label="Messured downlink speed when ping below {} ms".format(
        str(pingbound_network_test)))

fig_highlight_below_acceptable_network_speed = hv.Scatter(
    (df_speed_test["date"][
        df_speed_test["downstream"]/1000 < acceptable_network_speed],
     df_speed_test["downstream"][
         df_speed_test["downstream"]/1000 < acceptable_network_speed]/1000),
    "Date",
    "Network Speed",
    label="Highlight downstream speed below {} mbit/s".format(
        str(acceptable_network_speed))).opts(color="red", size=10)

fig_horizontal_marker = hv.HLine(
    acceptable_network_speed,
    label="Acceptable network speed at {} mbit/s".format(
        str(acceptable_network_speed))).opts(color="black")

fig_upstream_below_ping_bound = hv.Curve(
    (df_speed_test["date"], df_speed_test["upstream"]/1000),
    "Date",
    "Network Speed",
    label="Messured uplink when ping below {} ms".format(
        str(pingbound_network_test))).opts(color="purple")

fig_network_speeds_under_upper_bound = (
    fig_network_speed_below_pingbound *
    fig_highlight_below_acceptable_network_speed * fig_upstream_below_ping_bound* fig_horizontal_marker
).opts(
    title="Network Speed when Ping below {} ms".format(pingbound_network_test),
    legend_position="top_left",
    padding=0.05)

#Safe newly generated plot
hv.save(fig_network_speeds_under_upper_bound,
        os.path.join("webpage", "figures",
                     "fig_network_speeds_under_upper_bound.html"),
        backend='bokeh')
fig_network_speeds_under_upper_bound

In [24]:
df_ping["qcut"] = pd.qcut(df_ping["avg"],10)

In [25]:
df_ping.groupby("qcut").agg({"avg":["count"]}).reset_index()

In [26]:
# Test